In [1]:
import pandas as pd
import re
from tqdm import tqdm

In [2]:
file_path = "~/Downloads/fda_substances_filtered(NK edits - 10.18.csv" # adjust this

fda_approved_additives = pd.read_csv(file_path,skiprows=0,encoding='latin1').rename(columns={"Other Names":"other_names_cln","Used for (Technical Effect)":"used_for","UPF marker (cosmetic additive) as per Nova ":"is_upf_marker"})
fda_approved_additives['other_names_cln'] = fda_approved_additives['other_names_cln'].apply(eval)

fda_approved_additives['used_for'] = fda_approved_additives['used_for'].str.split(",<br />")
data_split = fda_approved_additives[['Substance','is_upf_marker','other_names_cln','used_for']].explode("other_names_cln").explode("used_for")

data_split['Substanc'] = data_split['Substance'].str.strip()
data_split['other_names_cln'] = data_split['other_names_cln'].str.strip().str.lower()
data_split['used_for'] = data_split['used_for'].str.strip()

In [3]:
data_split['len_names'] = data_split['other_names_cln'].str.len()
data_split = data_split.query("len_names>=3")

In [4]:
file_path = "kroger_2023-05-30final.csv"

df = pd.read_csv(file_path,low_memory=False)
df_ing = df.dropna(subset="ingredients")
strings = df_ing['ingredients'].str.lower().tolist()
mapping1 = dict(zip(data_split['other_names_cln'],data_split['used_for']))
mapping2 = dict(zip(data_split['other_names_cln'],data_split['Substance']))

# Build a regex pattern that matches any of the keys
pattern = r'\b(' + '|'.join(re.escape(key) for key in mapping1.keys()) + r')\b'
regex = re.compile(pattern)

result_usage = []
result_name = []
for s in tqdm(strings):
    # Find all matches in the string
    matches = regex.findall(s)
    
    # Map the matches to their corresponding values
    values2 = [mapping2[match] for match in matches]
    values2 = list(set(values2))
    result_name.append(values2)
    
    values1 = data_split.loc[data_split['Substance'].isin(values2),["Substance","used_for"]].drop_duplicates()['used_for'].tolist()
    result_usage.append(values1)
        
# Map the matches to their corresponding values
   

100%|███████████████████████████████████████████████████████████████████████████| 22095/22095 [01:14<00:00, 296.45it/s]


In [5]:
df_ing['additives_detected'] = result_name
df_ing['additives_usage'] = result_usage

C:\Users\vinso\AppData\Local\Temp\ipykernel_22024\2735500478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ing['additives_detected'] = result_name
C:\Users\vinso\AppData\Local\Temp\ipykernel_22024\2735500478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ing['additives_usage'] = result_usage


In [21]:
fda_approved_additives[['Substance','used_for','is_upf_marker']].explode("used_for")#.to_csv("30th_oct-additives_info.csv",index=False)

,Substance,used_for,is_upf_marker
0,ACESULFAME POTASSIUM,NON-NUTRITIVE SWEETENER,Yes
1,BETA-ALANINE,FLAVOR ENHANCER,Yes
1,BETA-ALANINE,FLAVORING AGENT OR ADJUVANT,Yes
1,BETA-ALANINE,NUTRIENT SUPPLEMENT,Yes
2,L-ALANINE,NUTRIENT SUPPLEMENT,No
...,...,...,...
495,MONOSODIUM GLUTAMATE,NUTRIENT SUPPLEMENT,Yes
495,MONOSODIUM GLUTAMATE,PH CONTROL AGENT,Yes
495,MONOSODIUM GLUTAMATE,STABILIZER OR THICKENER,Yes
496,MONOAMMONIUM GLUTAMATE,FLAVOR ENHANCER,Yes


In [20]:
df_ing[['upc','additives_detected']].explode("additives_detected").drop_duplicates().dropna()#.to_csv("30th_oct-additives_extracted.csv",index=False)

,upc,additives_detected
41,827466500,SUCROSE
41,827466500,CITRIC ACID
41,827466500,GINGER (ZINGIBER OFFICINALE ROSC.)
42,827466600,CITRIC ACID
42,827466600,GINGER (ZINGIBER OFFICINALE ROSC.)
...,...,...
47264,7088884900665,CALCIUM CARBONATE
47264,7088884900665,PSYLLIUM SEED HUSK
47264,7088884900665,CORNSTARCH
47286,9002531700696,SUCROSE
